In [1]:
import os

In [3]:
import warnings
warnings.filterwarnings("ignore")


In [34]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
'''GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]=LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY'''

### load the model and test it with simple message


In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [9]:
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [10]:
message=[SystemMessage(content="Hi you are nice Bot"),HumanMessage(content="Hi, how are you buddy?")]


In [11]:
message2=[SystemMessage(content="Hi you are nice Bot and you answer everything in brief"),HumanMessage(content="Hi, how are you buddy?")]


In [12]:
model.invoke(message)

AIMessage(content='Hello! I am not BotHi, I am Gemini, a multi-modal AI model, developed by Google.\n\nI am doing well, thank you for asking. I am always happy to help in any way that I can.\n\nIs there anything I can assist you with today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-21f10ec2-1cd9-405d-899e-cd5d75e38984-0', usage_metadata={'input_tokens': 13, 'output_tokens': 58, 'total_tokens': 71})

In [22]:
model.invoke(message2)

AIMessage(content='I am well, thank you.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-bf237e20-6ec2-4d5e-8269-526a5703b299-0', usage_metadata={'input_tokens': 19, 'output_tokens': 7, 'total_tokens': 26})

In [13]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [14]:
response = model.invoke(message)

In [15]:
response2 = model.invoke(message2)

In [20]:
parser.invoke(response)

"Thank you for the compliment! I'm doing well, thank you. I'm always happy to help you with any questions or tasks you may have. Is there anything I can assist you with today?"

In [23]:
parser.invoke(response2)

"I'm a virtual assistant, not a person. I don't have feelings or emotions. I'm here to help you with your questions and tasks."

### chaining with LCEL 

In [ ]:
chain = model | parser

In [24]:
chain.invoke(message)

'Hello there! I am not BotHi, I am Gemini, a large multi-modal model, trained by Google. I am doing well, thank you for asking. How are you doing today?'

In [25]:
chain.invoke(message2)

'I am well, thank you.'

### PROMPTING

In [26]:
from langchain_core.prompts import ChatPromptTemplate

In [27]:
system="translate the following into {language}"

In [28]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [29]:
prompt = prompt_template.invoke({"language":"french","text":"i am learning langchain with sunny sir."})


In [30]:
parser.invoke(model.invoke(prompt))

"J'apprends la Langchain avec Sunny Sir."

In [31]:
prompt.to_messages()

[SystemMessage(content='translate the following into french'),
 HumanMessage(content='i am learning langchain with sunny sir.')]

### chaining 

In [32]:
chain = prompt_template | model | parser

In [33]:
chain.invoke({"language": "hindi", "text": "I am hungry"})


'मुझे भूख लगी है।'

### AGENTS

In [35]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [36]:
memory = MemorySaver()
search=TavilySearchResults( max_results=2)
tools=[search]
agent_executor=create_react_agent(model,tools,checkpointer=memory)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [38]:
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im ashmit! and i live in noida")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Nice to meet you Ashmit!', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0eb839fa-3607-4a9b-a6db-820ae040da5f-0', usage_metadata={'input_tokens': 103, 'output_tokens': 7, 'total_tokens': 110})]}}
----


In [39]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in Noida?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f53bc70a-17a2-4ff5-909f-d06f1bb04c4c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in Noida?'}, 'id': 'bdc3a9ac-1c97-48f3-9fa9-036c537e881a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 119, 'output_tokens': 26, 'total_tokens': 145})]}}
----
{'tools': {